In [318]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [319]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.utils import np_utils
# from keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import class_weight
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', -1)

In [320]:
df = pd.read_csv('train.csv',sep=',')
df.drop(['Accident_ID'],inplace=True,axis=1)
# df['Combined'] = df['Total_Safety_Complaints'] + df['Turbulence_In_gforces'] + df['Cabin_Temperature'] + df['Max_Elevation'] + df['Violations'] + df['Adverse_Weather_Metric']
# df.drop(['Total_Safety_Complaints', 'Turbulence_In_gforces', 'Cabin_Temperature', 'Max_Elevation', 'Violations', 'Adverse_Weather_Metric'],inplace=True,axis=1)
test_df = pd.read_csv('test.csv',sep=',')
test_df_accident = test_df[['Accident_ID']]
test_df.drop('Accident_ID',inplace=True,axis=1)
# test_df['Combined'] = test_df['Total_Safety_Complaints'] + test_df['Turbulence_In_gforces'] + test_df['Cabin_Temperature'] + test_df['Max_Elevation'] + test_df['Violations'] + test_df['Adverse_Weather_Metric']
# test_df.drop(['Total_Safety_Complaints', 'Turbulence_In_gforces', 'Cabin_Temperature', 'Max_Elevation', 'Violations', 'Adverse_Weather_Metric'],inplace=True,axis=1)

df.head()


,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883


In [321]:
df.describe()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000
mean,41.876406,12.931100,6.564300,65.145324,0.381495,79.969331,3.814900,32001.803282,2.01220,0.255635
std,16.138072,3.539803,6.971982,11.882934,0.121301,2.759739,1.902577,9431.995196,1.03998,0.381128
min,0.000000,1.000000,0.000000,0.000000,0.134000,74.740000,1.000000,831.695553,0.00000,0.000316
25%,30.593607,11.000000,2.000000,56.927985,0.293665,77.960000,2.000000,25757.636908,1.00000,0.012063
50%,41.278539,13.000000,4.000000,65.587967,0.365879,79.540000,4.000000,32060.336419,2.00000,0.074467
75%,52.511416,15.000000,9.000000,73.336372,0.451346,81.560000,5.000000,38380.641513,3.00000,0.354059
max,100.000000,23.000000,54.000000,100.000000,0.882648,97.510000,7.000000,64297.651218,5.00000,2.365378


In [322]:
df['Severity'].value_counts()

Highly_Fatal_And_Damaging                  3049
Significant_Damage_And_Serious_Injuries    2729
Minor_Damage_And_Injuries                  2527
Significant_Damage_And_Fatalities          1695
Name: Severity, dtype: int64

In [323]:
severity_label_mapping = {'Highly_Fatal_And_Damaging': 3,
'Significant_Damage_And_Serious_Injuries': 2,
'Minor_Damage_And_Injuries': 0,
'Significant_Damage_And_Fatalities': 1
}
df["Severity"] = df["Severity"].map(severity_label_mapping).astype(np.int32)


In [324]:
df.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
0,0,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352
1,0,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350
2,1,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364
3,2,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728
4,1,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883


In [325]:
# def min_max(x,minx,maxx):
#     return (x-minx)/(maxx-minx)

# def standardscale(x,meanx,stdx):
#     return (x-meanx)/stdx

# df['Safety_Score'] = df['Safety_Score'].apply(standardscale, args=(df['Safety_Score'].mean(),df['Safety_Score'].std()))
# df['Max_Elevation'] = df['Max_Elevation'].apply(standardscale, args=(df['Max_Elevation'].mean(),df['Max_Elevation'].std()))
# df['Control_Metric'] = df['Control_Metric'].apply(standardscale, args=(df['Control_Metric'].mean(),df['Control_Metric'].std()))
# df['Days_Since_Inspection'] = df['Days_Since_Inspection'].apply(min_max, args=(df['Days_Since_Inspection'].min(),df['Days_Since_Inspection'].max()) )
# df['Total_Safety_Complaints'] = df['Total_Safety_Complaints'].apply(min_max, args=(df['Total_Safety_Complaints'].min(),df['Total_Safety_Complaints'].max()) )
# df['Turbulence_In_gforces'] = df['Turbulence_In_gforces'].apply(min_max, args=(df['Turbulence_In_gforces'].min(),df['Turbulence_In_gforces'].max()) )
# df['Cabin_Temperature'] = df['Cabin_Temperature'].apply(min_max, args=(df['Cabin_Temperature'].min(),df['Cabin_Temperature'].max()) )
# df['Accident_Type_Code'] = df['Accident_Type_Code'].apply(min_max, args=(df['Accident_Type_Code'].min(),df['Accident_Type_Code'].max()) )
# df['Violations'] = df['Violations'].apply(min_max, args=(df['Violations'].min(),df['Violations'].max()) )
# df['Adverse_Weather_Metric'] = df['Adverse_Weather_Metric'].apply(min_max, args=(df['Adverse_Weather_Metric'].min(),df['Adverse_Weather_Metric'].max()) )

# df.head()

In [326]:
df.describe()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000
mean,1.630000,41.876406,12.931100,6.564300,65.145324,0.381495,79.969331,3.814900,32001.803282,2.01220,0.255635
std,1.161221,16.138072,3.539803,6.971982,11.882934,0.121301,2.759739,1.902577,9431.995196,1.03998,0.381128
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.134000,74.740000,1.000000,831.695553,0.00000,0.000316
25%,0.000000,30.593607,11.000000,2.000000,56.927985,0.293665,77.960000,2.000000,25757.636908,1.00000,0.012063
50%,2.000000,41.278539,13.000000,4.000000,65.587967,0.365879,79.540000,4.000000,32060.336419,2.00000,0.074467
75%,3.000000,52.511416,15.000000,9.000000,73.336372,0.451346,81.560000,5.000000,38380.641513,3.00000,0.354059
max,3.000000,100.000000,23.000000,54.000000,100.000000,0.882648,97.510000,7.000000,64297.651218,5.00000,2.365378


In [327]:
df.columns

Index(['Severity', 'Safety_Score', 'Days_Since_Inspection',
       'Total_Safety_Complaints', 'Control_Metric', 'Turbulence_In_gforces',
       'Cabin_Temperature', 'Accident_Type_Code', 'Max_Elevation',
       'Violations', 'Adverse_Weather_Metric'],
      dtype='object')

In [328]:
x_columns = ['Safety_Score', 'Days_Since_Inspection', 'Total_Safety_Complaints', 'Control_Metric',
             'Turbulence_In_gforces', 'Cabin_Temperature', 'Accident_Type_Code',
             'Max_Elevation', 'Violations', 'Adverse_Weather_Metric']

y_columns = ['Severity']

In [329]:
X = df.drop(['Severity'], axis = 1) 
Y = df[['Severity']]

X.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
0,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352
1,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350
2,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364
3,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728
4,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883


In [330]:
# x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=0,stratify=Y)


In [331]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)



In [332]:
class_weights = list(class_weight.compute_class_weight('balanced', np.unique(Y['Severity']), Y['Severity']))

w_array = np.ones(Y.shape[0], dtype = 'float')
for i, val in enumerate(Y.values):
    w_array[i] = class_weights[int(val)-1]



In [333]:

# clfxgb = XGBClassifier(eta = 0.01, max_depth=10, objective='multi:softmax', n_estimators=1000, 
#                         num_classes=4, random_state=0, n_jobs=3, sample_weight=w_array)
# clfxgb.fit(X,Y)

# for foo,bar in zip(x_columns,clfxgb.feature_importances_):
#     print(foo,bar*100)
    

In [335]:
clfrf = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_split=4, n_jobs=-1,class_weight="balanced_subsample").fit(X, Y)


/Users/madhur/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [336]:

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 4,
    'metric': 'multi_logloss',
    'n_jobs': -1,
    'num_leaves': 40,
    'learning_rate': 1
}
lgb_train = lgb.Dataset(X,Y)
clfgbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000)


In [337]:
# model_keras = Sequential()
# model_keras.add(Dense(8, input_dim=10, activation='relu'))
# model_keras.add(Dense(7, activation='relu'))
# model_keras.add(Dense(6, activation='relu'))
# model_keras.add(Dense(5, activation='relu'))
# model_keras.add(Dense(4, activation='softmax'))
# # Compile model
# model_keras.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model_keras.fit(X, to_categorical(Y),batch_size=10,epochs=100,verbose=1,shuffle=True)



In [338]:
# test_df['Safety_Score'] = test_df['Safety_Score'].apply(standardscale, args=(test_df['Safety_Score'].mean(),test_df['Safety_Score'].std()))
# test_df['Max_Elevation'] = test_df['Max_Elevation'].apply(standardscale, args=(test_df['Max_Elevation'].mean(),test_df['Max_Elevation'].std()))
# test_df['Control_Metric'] = test_df['Control_Metric'].apply(standardscale, args=(test_df['Control_Metric'].mean(),test_df['Control_Metric'].std()))
# test_df['Days_Since_Inspection'] = test_df['Days_Since_Inspection'].apply(min_max, args=(test_df['Days_Since_Inspection'].min(),test_df['Days_Since_Inspection'].max()) )
# test_df['Total_Safety_Complaints'] = test_df['Total_Safety_Complaints'].apply(min_max, args=(test_df['Total_Safety_Complaints'].min(),test_df['Total_Safety_Complaints'].max()) )
# test_df['Turbulence_In_gforces'] = test_df['Turbulence_In_gforces'].apply(min_max, args=(test_df['Turbulence_In_gforces'].min(),test_df['Turbulence_In_gforces'].max()) )
# test_df['Cabin_Temperature'] = test_df['Cabin_Temperature'].apply(min_max, args=(test_df['Cabin_Temperature'].min(),test_df['Cabin_Temperature'].max()) )
# test_df['Accident_Type_Code'] = test_df['Accident_Type_Code'].apply(min_max, args=(test_df['Accident_Type_Code'].min(),test_df['Accident_Type_Code'].max()) )
# test_df['Violations'] = test_df['Violations'].apply(min_max, args=(test_df['Violations'].min(),test_df['Violations'].max()) )
# test_df['Adverse_Weather_Metric'] = test_df['Adverse_Weather_Metric'].apply(min_max, args=(test_df['Adverse_Weather_Metric'].min(),test_df['Adverse_Weather_Metric'].max()) )

test_df = scaler.transform(test_df)

In [339]:
#xgb classifier
# y_test_xgb = clfxgb.predict(test_df)

# keras
# y_test_keras = model_keras.predict(test_df)
# y_test_keras_np = np.argmax(y_test_keras, axis=1)

#randomforrest
y_test_rf = clfrf.predict(test_df)

#lgbm
y_test_lgbm = clfgbm.predict(test_df, num_iteration=clfgbm.best_iteration)
y_test_lgbm = np.argmax(y_test_lgbm, axis=1)


In [340]:
# test_df_accident['Severity_xgb'] = y_test_xgb

# test_df_accident['Severity_keras'] = y_test_keras_np
test_df_accident['Severity_rf'] = y_test_rf
test_df_accident['Severity_lgbm'] = y_test_lgbm

test_df_accident.head(30)

,Accident_ID,Severity_rf,Severity_lgbm
0,1,3,3
1,10,1,1
2,14,2,2
3,17,3,3
4,21,1,1
5,40,1,1
6,45,1,1
7,50,3,3
8,52,3,3
9,85,2,2


In [341]:
# test_df_accident['Severity_Total'] = test_df_accident.loc[:,['Severity_xgb',
#                                                              'Severity_rf','Severity_lgbm']].sum(axis=1)

# test_df['severity_Total'] = test_df.iloc[:, 1:5].sum(axis=1)
# test_df_accident['Severity_Total'] = test_df_accident['Severity_Total']/3.0
# test_df_accident['Severity_Total'] = test_df_accident['Severity_Total'].astype(int)
test_df_accident.head(500)

,Accident_ID,Severity_rf,Severity_lgbm
0,1,3,3
1,10,1,1
2,14,2,2
3,17,3,3
4,21,1,1
5,40,1,1
6,45,1,1
7,50,3,3
8,52,3,3
9,85,2,2


In [342]:
# from collections import Counter
# def fun(x):
#     x = list(x.values)
#     data = Counter(x)
#     return data.most_common(1)[0][0]
# test_df_accident['Severity_Mode'] = test_df_accident[['Severity_xgb','Severity_rf','Severity_lgbm']].apply(fun,axis=1)
# test_df_accident.head()

In [343]:
severity_label_mapping_reverse = {3:'Highly_Fatal_And_Damaging',
                                  2:'Significant_Damage_And_Serious_Injuries',
                                  0:'Minor_Damage_And_Injuries',
                                  1:'Significant_Damage_And_Fatalities'}

test_df_accident["Severity_lgbm"] = test_df_accident["Severity_lgbm"].map(severity_label_mapping_reverse)
# test_df_accident["Severity_xgb"] = test_df_accident["Severity_xgb"].map(severity_label_mapping_reverse)
# test_df_accident["Severity_keras"] = test_df_accident["Severity_keras"].map(severity_label_mapping_reverse)
test_df_accident["Severity_rf"] = test_df_accident["Severity_rf"].map(severity_label_mapping_reverse)
# test_df_accident["Severity_Mode"] = test_df_accident["Severity_Mode"].map(severity_label_mapping_reverse)

test_df_accident.head(10)



,Accident_ID,Severity_rf,Severity_lgbm
0,1,Highly_Fatal_And_Damaging,Highly_Fatal_And_Damaging
1,10,Significant_Damage_And_Fatalities,Significant_Damage_And_Fatalities
2,14,Significant_Damage_And_Serious_Injuries,Significant_Damage_And_Serious_Injuries
3,17,Highly_Fatal_And_Damaging,Highly_Fatal_And_Damaging
4,21,Significant_Damage_And_Fatalities,Significant_Damage_And_Fatalities
5,40,Significant_Damage_And_Fatalities,Significant_Damage_And_Fatalities
6,45,Significant_Damage_And_Fatalities,Significant_Damage_And_Fatalities
7,50,Highly_Fatal_And_Damaging,Highly_Fatal_And_Damaging
8,52,Highly_Fatal_And_Damaging,Highly_Fatal_And_Damaging
9,85,Significant_Damage_And_Serious_Injuries,Significant_Damage_And_Serious_Injuries


In [344]:
import time
t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)

test_df_accident.to_csv(f'output_lgbm_{timestamp}.csv',index=False,columns=['Accident_ID','Severity_lgbm'])
# test_df_accident.to_csv(f'output_xgb_{timestamp}.csv',index=False,columns=['Accident_ID','Severity_xgb'])
# test_df_accident.to_csv(f'output_keras_{timestamp}.csv',index=False,columns=['Accident_ID','Severity_keras'])
test_df_accident.to_csv(f'output_rf_{timestamp}.csv',index=False,columns=['Accident_ID','Severity_rf'])



In [345]:
# test_df_accident.to_csv(f'output_mode_{timestamp}.csv',index=False,columns=['Accident_ID','Severity_Mode'])


In [346]:
df.describe()



,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000
mean,1.630000,41.876406,12.931100,6.564300,65.145324,0.381495,79.969331,3.814900,32001.803282,2.01220,0.255635
std,1.161221,16.138072,3.539803,6.971982,11.882934,0.121301,2.759739,1.902577,9431.995196,1.03998,0.381128
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.134000,74.740000,1.000000,831.695553,0.00000,0.000316
25%,0.000000,30.593607,11.000000,2.000000,56.927985,0.293665,77.960000,2.000000,25757.636908,1.00000,0.012063
50%,2.000000,41.278539,13.000000,4.000000,65.587967,0.365879,79.540000,4.000000,32060.336419,2.00000,0.074467
75%,3.000000,52.511416,15.000000,9.000000,73.336372,0.451346,81.560000,5.000000,38380.641513,3.00000,0.354059
max,3.000000,100.000000,23.000000,54.000000,100.000000,0.882648,97.510000,7.000000,64297.651218,5.00000,2.365378


In [347]:
test_df_accident.head()

,Accident_ID,Severity_rf,Severity_lgbm
0,1,Highly_Fatal_And_Damaging,Highly_Fatal_And_Damaging
1,10,Significant_Damage_And_Fatalities,Significant_Damage_And_Fatalities
2,14,Significant_Damage_And_Serious_Injuries,Significant_Damage_And_Serious_Injuries
3,17,Highly_Fatal_And_Damaging,Highly_Fatal_And_Damaging
4,21,Significant_Damage_And_Fatalities,Significant_Damage_And_Fatalities


In [348]:
# mostCommonVote=[]
# for row in test_df_accident[['Severity_xgb', 'Severity_lgbm']].values:
#     votes, values = np.unique(row, return_counts=True)
#     if np.all(values<=1):
#             mostCommonVote.append( row[0] )
#     else:
#         mostCommonVote.append( votes[np.argmax(values)] )

# test_df_accident['Severity_majority'] = mostCommonVote


In [349]:
# test_df_accident.to_csv(f'output_majority_two_{timestamp}.csv',index=False,columns=['Accident_ID','Severity_majority'])


In [350]:
df['Violations'].unique()

array([3, 2, 1, 0, 4, 5])